In [2]:
import pandas as pd
import json
def CustomParser(data):
    j1 = json.loads(data)
    return j1
from pandas import Series, DataFrame
import numpy as np
from numpy import array
import missingno as mn
import re
import datetime
import time
import itertools 
from itertools import chain, combinations, product
from collections import Counter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

from pandas import DataFrame, Series
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy
from collections import Counter
import math
import copy

pd.set_option('display.max_columns', 500) #column printing
pd.set_option('display.max_rows', 100) #row printing
pd.options.display.float_format = '{:,.3f}'.format #Limit output to 3 decimal places

from IPython.display import display

import plotly.graph_objs as go

D:\work\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# this first portion of the app structures the page 
# the page is meant to display a series of dropdowns and tables
# one dropdown and one table are provided per SQL prompt

app = dash.Dash()
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(dt.DataTable(rows=[{}]),
             style={'display': 'none'}),
    html.Div(id='page-content')
])
app.config['suppress_callback_exceptions']=True

# designing page layout below
index_page = html.Div([
    dcc.Link('Schoolytics', href='/page-1'),
    html.Br(),
    dcc.Dropdown(
    id='student_dd1',
    options=[{}],
    placeholder='Select Student',
    multi=False,
    value=None,
),
    html.Div([                    
        dt.DataTable(
        rows=[{}],
        selected_row_indices=[],
        max_rows_in_viewport=15,
        id='query1'
        ),
    dcc.Dropdown(
    id='course_dd1',
    options=[{}],
    placeholder='Select Course',
    multi=False,
    value=None,
),
    html.Div([                    
        dt.DataTable(
        rows=[{}],
        selected_row_indices=[],
        max_rows_in_viewport=15,
        id='query2'
        ),
    dcc.Dropdown(
    id='teacher_dd1',
    options=[{}],
    placeholder='Select Teacher',
    multi=False,
    value=None,
),
    html.Div([                    
        dt.DataTable(
        rows=[{}],
        selected_row_indices=[],
        max_rows_in_viewport=15,
        id='query3'
        ),
    dcc.Dropdown(
    id='school_dd1',
    options=[{}],
    placeholder='Select School',
    multi=False,
    value=None,
),
    html.Div([                    
        dt.DataTable(
        rows=[{}],
        selected_row_indices=[],
        max_rows_in_viewport=15,
        id='query4'
        ),
        
],className = 'ten columns offset-by-one')
################################################
        
### 
# the callbacks below define the behavior of the features displayed on this page
###
        
# 'student_dd1' creates the set of students available for selection for the first query
# "All Students" is also available as a selection, which outputs a full table of students
@app.callback(dash.dependencies.Output('student_dd1', 'options'),
              [])
def student_options():  
    # this creates the dropdown options for query1
    db_connection = create_engine(credentials,echo=False)
    school=pd.read_sql('select unique(student_id) from student_submission',db_connection)
    student_options={s:s for s in school}
    student_options.extend([{'label':'All Students','value':'All Students'}])
    return school_options
    
        
# this callback creates a Plotly table that displays all students and counts their completed assignments
# it accepts one input of 'student' and filters down to a single row if a student is selected
@app.callback(dash.dependencies.Output('query1', 'rows'),
              [dash.dependencies.Input('student_dd1', 'value')])
def student_assignment(student):
    # group by student id and count completed assignments
    db_connection = create_engine(credentials,echo=False)
    if student=='All Students':
        school=pd.read_sql('''select
                                student_id,
                                count(status)
                            from student_submission 
                            where status = 'TURNED_IN'
                            group by 
                                student_id 
                            order by 
                                count(status) 
                                desc''',
                          db_connection)
    else:
        school=pd.read_sql('''select
                                student_id,
                                count(status)
                            from student_submission 
                            where status = 'TURNED_IN'
                            and student_id= %(stu_id)s
                            group by 
                                student_id 
                            order by 
                                count(status) 
                                desc''',
                          db_connection,params={'stu_id':student})

    return school.to_dict('records') 
    
        
# this callback creates the set of courses available for selection for the second query
@app.callback(dash.dependencies.Output('course_dd1', 'options'),
              [])
def course_options():  
    # this creates the dropdown options for query2
    db_connection = create_engine(credentials,echo=False)
    school=pd.read_sql('select unique(course_id) from student_submission',db_connection)
    course_options={s:s for s in school}
    course_options.extend([{'label':'All Courses','value':'All Courses'}])
    return course_options
    
# this callback creates a Plotly table that displays all courses and counts their average grade
@app.callback(dash.dependencies.Output('query2', 'rows'),
              [dash.dependencies.Input('course_dd1', 'value')])
def average_grades(course):
    # group by course_id and average grade
    db_connection = create_engine(credentials,echo=False)
    if course=='All Courses':
        school=pd.read_sql('''select 
                                course_id,
                                avg(assigned_points/max_points) as point_avg,
                                status 
                            from student_submission 
                            where status = 'TURNED_IN' 
                            and max_points !=0
                            group by 
                                course_id
                            order by 
                                avg(assigned_points/max_points) 
                                desc''',
                          db_connection)
    else:
        school=pd.read_sql('''select 
                                course_id,
                                avg(assigned_points/max_points) as point_avg,
                                status 
                            from student_submission 
                            where status = 'TURNED_IN' 
                            and max_points !=0
                            and course_id= %(c_id)s 
                            group by 
                                course_id
                            order by 
                                avg(assigned_points/max_points) 
                                desc''',
                          db_connection,params={'c_id':course})
    return school.to_dict('records')
    
# this callback creates the set of teachers available for selection for the third query 
@app.callback(dash.dependencies.Output('teacher_dd1', 'options'),
              [])
def teacher_options():  
    # this creates the dropdown options for query3
    db_connection = create_engine(credentials,echo=False)
    school=pd.read_sql('select unique(teacher_id) from course',db_connection)
    teacher_options={s:s for s in school}
    teacher_options.extend([{'label':'All Teachers','value':'All Teachers'}])
    return course_options
    
        
# this callback creates a Plotly table that displays all teachers and counts their assigned work
@app.callback(dash.dependencies.Output('query3', 'rows'),
              [dash.dependencies.Input('teacher_dd1', 'value')])
def average_grades(teacher):
    # group by course_id and average grade
    db_connection = create_engine(credentials,echo=False)
    
    if teacher=='All Teachers':
        school=pd.read_sql('''select 
                                count(ss.course_work_id) 
                            from student_submission as ss
                            right join course as c on
                                ss.course_id=c.course_id
                            group by 
                                teacher_id''',
                          db_connection)
    else:
    
        school=pd.read_sql('''select 
                                count(ss.course_work_id) 
                            from student_submission as ss
                            right join course as c on
                                ss.course_id=c.course_id
                            where teacher_id = %(teach_id)s
                            group by 
                                teacher_id''',
                          db_connection,params={'teach_id':teacher})

    return school.to_dict('records')
    
# this callback creates the set of schools available for selection for the fourth query 
@app.callback(dash.dependencies.Output('school_dd1', 'options'),
              [])
def school_options():  
    # this creates the dropdown options for query4
    db_connection = create_engine(credentials,echo=False)
    school=pd.read_sql('select UNIQUE(school_id) from student',db_connection)
    school_options={s:s for s in school}
    school_options.extend([{'label':'All Schools','value':'All Schools'}])
    return school_options
    
        
# this callback creates a Plotly table that displays all schools and shows the percentage of students with at least one assignment done
@app.callback(dash.dependencies.Output('query4', 'rows'),
              [dash.dependencies.Input('school_dd1', 'value')])
def percentage_of_students(school):
    #   Group by school_id and find percentage of students with more than 1 assignment completed
    db_connection = create_engine(credentials,echo=False)
    
    if school=='All schools':
        school=pd.read_sql('''with one_students as
                            (
                                select 
                                    distinct
                                    studentid,
                                    1 as turned_in_something
                                from student_submission
                                where status = 'TURNED IN'
                            )
                            select 
                                schoolid,
                                avg(coalesce(turned_in_something,0)) as pct_turned_in_something
                            from student as st
                            left join one_students as os
                                on os.studentid = st.studentid
                            group by
                                schoolid''',
                           db_connection)

    else:
        school=pd.read_sql('''with one_students as
                            (
                                select 
                                    distinct
                                    studentid,
                                    1 as turned_in_something
                                from student_submission
                                where status = 'TURNED IN'
                            )
                            select 
                                schoolid,
                                avg(coalesce(turned_in_something,0)) as pct_turned_in_something
                            from student as st
                            left join one_students as os
                                on os.studentid = st.studentid
                            where teacher_id = %(teach_id)s
                            group by
                                schoolid''',
                           db_connection,params={'teach_id':teacher})

    return school.to_dict('records')
    

@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-1':
        return page_1_layout        
    else:
        return page_1_layout


if __name__ == '__main__':
    app.run_server(debug=False)